# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [18]:
browser = webdriver.Chrome(executable_path='../DAY18/chromedriver',chrome_options=chrome_options)
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        print('到達頁面底端')
        break
    else:
        last_height = new_height
        print('網頁更新中...')

/tmp/ipykernel_102/2484200703.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path='../DAY18/chromedriver',chrome_options=chrome_options)
/tmp/ipykernel_102/2484200703.py:1: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(executable_path='../DAY18/chromedriver',chrome_options=chrome_options)


網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [19]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [20]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 塞港蔓延到地中海、南非等港　估計明年年中才能緩解

▲美西塞港問題已蔓延到地中海與南非等地。（圖／翻攝自寰宇新聞台）
記者張佩芬／台北報導
全球貨櫃港塞港問題今年一整年困擾著託運人與運送人，也造成供應鏈危機，丹麥貨櫃船運咨詢公司eeSea近日發布的數據顯示，塞港已從南加州蔓延到地中海、南非與馬來西亞的巴生港等，不過相關業者表示，巴生港是因為連日下大雨造成塞港，應該很快可以貨獲得改善。
eeSea的數據顯示，巴生港正遭受嚴重擁堵，有32艘貨櫃船等待泊位。瀕臨地中海的北非阿爾及利亞港和希臘的比雷埃夫斯港則是塞港日益嚴重。
國內海運業者指出，比雷埃夫斯港是地中海地主要轉口港，本月因為碼頭工人曾經發動48小時罷工，加上冬季氣候不佳，引水人無法正常出海領港，造成塞港惡化，貨櫃場也變得十分擁擠。一個港出問題，鄰近其他港也跟這亂套，目前候港時間要五天。
巴生港因為是東南亞主要轉口港，進出貨量很大，託運人可視性平台Project44上個月的一份報告顯示，今年前10個月，該港口經歷的空白航行增加了980%。
塞港最嚴重的洛杉磯/長灘港，雖然很努力的在緩解陸側交通的擁堵雨貨櫃積壓問題，但是本月15日多達101艘等候靠港裝卸的船隻，總運力約80萬箱(20呎櫃)，讓情況無法樂觀看待。物流專家表示，美國進口貨量曆史性的激增，讓目前積壓的貨櫃估計要到明年年中才能清理完畢。業者則提醒，前提是美國西岸碼頭工人換約談判要很順利，沒有出現嚴重怠工或罷工事件發生。
業者估計，目前積壓的貨物需要五個多月才能清理完畢，但是新的貨還在持續運進，美國進口商會搶在春季進口旺季前與碼頭工人可能發動的怠工、罷工時點前進貨，農曆年過後美國港口馬上面臨新一波貨運浪潮。
----------------------------------------------------------------------

[1] 快訊／耶誕前血洗！美國三大指數開盤均跌！道瓊下殺459點

記者鄒鎮宇／綜合報導
美股20日延續上週五的跌勢，三大指數開盤均下跌。台灣時間20日晚間10時30分開盤，道瓊工業指數下跌459.07點，跌幅1.30%，報34906.37點；標普500指數下跌57

▲歐股早盤全數翻黑。（圖／路透社）
記者陳心怡／綜合報導
美股三大期貨指數今（20）日上午開始就出現下跌態勢，截至17時7分為止，道瓊期指大跌超過600點，而那斯達克100期與標普500期指也都出現下跌，跌幅皆超過1.8%。歐洲股市今日早盤同步走低，德國DAX指數、法國CAC 40指數下跌逾2％。
截至台灣時間下午5點35分，歐股表現如下：
----------------------------------------------------------------------

[15] 高雄凹子底蓋好「巨蛋光環不保」？　在地人提2關鍵：不會沒落

記者黃可昀／綜合報導
高雄漢神巨蛋周邊是熱門地段，新建案已經站上4字頭。有網友近期想在巨蛋附近買房，看中一間1200萬中古三房，不過富邦凹子底商場預計2025年會完工，旁邊還有義享天地，這樣是否會影響到巨蛋周邊的房價，害他之後要換房時反而價跌？
原PO在PTT表示，他看中的這間三房開價1200萬，含平面車位，雖然以高雄的房價來說已經很貴，不過他看在這裡是蛋黃區，交通方便、生活機能好，也有學區，仍具有上漲潛力，想買來投資兼自住，不過將來有換屋打算。
由於原PO之後可能會換屋，所以房子的未來發展性就很重要，現在巨蛋站屬於精華地段，但3年後凹子底商場完工，加上今年剛蓋好的義享天地，可能會搶走很多人潮錢潮，導致他家價格下跌，轉手反而還虧，想問當地人「高雄漢神巨蛋週邊未來會沒落嗎？」
在地人認為，巨蛋和凹子底距離不遠，都是在同個生活圈，沒有人潮互搶的問題，完工後吸引的人潮也連帶讓巨蛋受益，「兩個商圈離這麼近，根本是是共同商圈怎麼比」、「凹子底跟巨蛋是連在一起的；不太可能那一頭漲翻天，這一頭就沒落沒人要」、「這兩邊是有差多遠？」
此外，網友也提到巨蛋有動線規劃好、交通方便等優勢，即使商圈轉移也至少要10年，原PO短時間內不用擔心巨蛋被取代，「巨蛋附近除生活機能強外，交通上國10、高鐵、捷運都很方便」、「以好逛程度，漢神巨蛋，不會沒落」、「高鐵三越開了多久，你覺得人潮有巨蛋一半嗎？」
----------------------------------------------------------------------

[16] 美股三大期貨持續下殺！道瓊期大跌逾600點、預告今晚美股開盤不妙

▲美股三大期貨持續下跌。（圖

▲國泰投信總經理張雍川。（圖／記者陳心怡攝）
記者陳心怡／台北報導
----------------------------------------------------------------------

[25] 新台幣收盤貶值6分　收在27.84元

▲新台幣收盤收在27.84元。（圖／記者湯興漢攝）
記者陳心怡／台北報導
台北外匯市場新台幣兌美元匯率今（20）日收盤收在27.84元，貶值6分。
新台幣今日以27.8元兌1美元開出，貶2分，盤中貶幅持續擴大，終場貶值6分，收在27.84元。
新台幣兌美元盤中最高27.795元，最低27.86元兌1美元。
----------------------------------------------------------------------

[26] 11月外銷訂單連21紅、創「歷年單月新高！」　全年突破6600億美元

記者林淑慧／台北報導
經濟部統計處今（20）日公布11月外銷訂單金額為655.0億美元，創下歷年單月新高，年增13.4%，不僅創下「連21紅」紀錄，更創歷年單月新高。統計處長黃于玲預測，今年全年外銷訂單可望年突破6600億美元。
依據統計，今年前十一月累計外銷訂單金額達6062.3億美元，突破6000億大關，為歷年同期新高；另，今年前十一月接單年增28.1%，為自1988年以來歷年同期年增最高。
▲經濟部統計處今（20）日公佈11月外銷訂單金額為655億美元，創歷年單月新高。（圖／經濟部提供）
依據統計處預測，12月外銷訂單金額為605至620億美元，月減7.6%至月減5.3%，年減0.1%至年增2.4%，有望拿下連22紅。統計處若預估準確，更有機會今年全年突破6600億美元。
黃于玲指出，資通信產品及電子產品受惠於5G、高效能運算等商機持續，及年底備貨需求釋出，推升外銷訂單持續繳出亮眼成績單。依照過往趨勢分析，要突破6600億美元大關有機會。但她強調，因為年底不確定因素多，還是必須等結果出爐。
依貨品別觀察，受惠5G、高效能運算等商機持續，加上年終銷售旺季， 以及筆電、顯示卡、伺服器等需求熱絡，帶動相關供應鏈接單增加，支撐電子及資訊通信產品接單成長，分別年增17.9%及1.7%；光學器材因商用、工控以及醫療等產業面板需求強勁，加上背光模組接單續增，惟大尺寸電視面板因價格下跌，抵銷部分

▲高鐵彰化站、中科彰化二林的效應加成，南彰化房市備受注目。（圖／記者謝婷婷攝）
記者陳筱惠／彰化報導
彰化縣田中、社頭、田尾、永靖等4鎮，受到高鐵彰化站2015年正式啟用、中科彰化二林的效應加成，整個南彰化房市備受注目，以田中來說，遠雄建設以總價1.2億元，拿下1180坪高鐵站住宅區土地引發關注。在地仲介認為，田中房市明年會將進入重頭戲。
《地產詹哥老實說》S2EP43／破解信義計畫區「刑場傳說」！　他催生全球最密集百貨街區
▲田中房價相對於員林市、北斗鎮等來說，還是具有基期低的優勢。（圖／東森房屋田中加盟店提供）
林縈潔分析，自彰化高鐵開通、中科二林帶動人口移入後，不少大型建商紛紛搶進南彰化推案，如遠雄建設、麗寶機構等，以目前新屋的行情，透天新案落在1200萬元上下、公寓新案產品總價落在600萬~700萬元。
而主力買盤則明顯分成2種，其一是首購族、小家庭，不論是在地客還是就業人口的移入，切中總價優勢而找尋公寓型產品；另一種則是退休族因喜歡「好山好水好空氣」的環境而在當地找尋適合物件。
▲東森房屋田中加盟店店東林縈潔分析，人口紅利帶動彰化西南角發展，整體房市發展正向、均衡。（圖／東森房屋田中加盟店提供）
林縈潔說田中房價相對於員林市、北斗鎮等來說，還是具有基期低的優勢；在建商進軍高鐵重劃區後，預計明年新屋供給量會增加；在需求部分從二林園區傳出矽品進駐後，人口紅利也帶動彰化西南角發展，整體房市發展正向、均衡。
《地產詹哥老實說》S2EP43／破解信義計畫區「刑場傳說」！　他催生全球最密集百貨街區
----------------------------------------------------------------------

[38] 光洋科經營權戰延燒　兩外資投票機構各挺一方

▲光洋科公司派代表馬堅勇。（資料照／記者吳康瑋攝）
記者吳康瑋／綜合報導
光洋科（1785）經營權大戰未平，兩大國際外資投票顧問機構 Glass Lewis 與 ISS 分別表態，其中，Glass Lewis 支持公司派馬堅勇團隊、ISS 則支持大股東台鋼集團。
公司派代表馬堅勇團隊指出，近日國際外資投票顧問機構Glass Lewis發布報告支持馬堅勇經營團隊。外資期盼光洋科經營權爭議需落實股東行動主義，透過股臨會投票選出新經營團隊讓紛爭落幕。
而市場派台鋼則在今（20）日指出

新聞節目中心／綜合報導
2021年台股表現亮眼，漲幅在全球主要市場中名列前茅，但面對2022更多不確定的因素，選產業和選股就變得極為重要。明年仍是半導體、電動車、元宇宙、虛擬貨幣當道，但細分到次產業，有哪些市場需求以及獲利成長前景皆為強勁的個股？12月20日《雲端最有錢》請到資深分析師林友銘、產業隊長張捷、內部籌碼達人老簡，要給觀眾最精準的產業和選股分析。
美國聯準會（FED）於15日舉行今年最後一次FOMC會議，會中刪除通膨是「暫時性」的措辭，說明美國通貨膨脹居高不下，已經成為首要經濟威脅。聯準會甚至還宣布，明年一月起把疫情後支撐經濟的縮減購債計畫金額，擴大到每個月300億美元，預估明年將會升息至少三次，態度明顯轉鷹派。
對於聯準會的舉動，美國股市先是利空出盡般的大漲，但隨即在週五出現崩跌，尤其是科技類股指數，這波直接減輕台股上攻18000點大關的力道。而QE退場對未來台股影響會很大嗎？除了升息疑慮，中長期的通膨也是一個不確定的因素，台股2022年該走向哪裡？
高手看籌碼！除了跟著外資和投信走，其實還有一個更精準的指標，那就是各公司的「內部大戶」，他們又比外資和投信更懂得內部公司運作，所以更能替公司「掛保證」；而這項資訊是極容易查得到，簡簡單單就能掌握內部大戶籌碼動向，勝率還高達九成，再搭配基本面分析和報表研讀「基本面為盾，籌碼面為矛」，就可以輕鬆抓住30%波段報酬率。節目中即將公開這獨門高勝率心法，所有精彩內容請鎖定《雲端最有錢》！
----------------------------------------------------------------------

[48] 「金飯碗」年薪大公開！6家金融證券業年薪連3年成長　這家暴增逾50%

▲近3年這6家金融證券業「非主管」員工平均年薪「連年成長」。（示意圖／記者陳依旻製表）
記者陳依旻／台北報導
歲末年終，不少上班族準備換工作，金融證券業也把握時機，紛紛公佈徵才訊息。檢視金融證券業近年薪資福利，至少有6家非擔任主管職務的員工「平均年薪」，是逐年成長的，《ETtoday新聞雲》也特別整理了相關情報資訊，讓大家參考瞭解「金飯碗」大概的市場行情。
金融證券業年薪誘人　前十名都破百萬元
根據證交所最新公佈的2020年「並未擔任主管」的全職員工，平均薪資前10名的金融保險證券業，依序為華票（150.4萬）

▲經濟部長王美花今（20）日出席立法院經濟委員會，回應國內缺電議題。（圖／記者屠惠剛攝）
記者林淑慧／台北報導
針對鴻海集團創辦人郭台銘對於公投後供電的擔憂，經濟部長王美花今（20）日表示，三接外推已有配套方案，經濟部會持續督促台電讓大潭8、9號機組如期完工，將現有氣源優先供氣給較高效率的燃氣機組，以穩定國內供電。
立法院經濟委員會今日審查經濟部及所屬單位預算案，邀請經濟部長王美花與會討論，而公投結果、國內缺電、台灣爭取加入CPTPP等議題，仍是各界關注焦點。
繼先前在兩岸企業家峰會上預警台灣電力供應問題，郭台銘昨日對公投結果表示，「明年是一定缺電，大家要有心理準備。」工業也會缺電，家庭也會缺電，既然大家做了這個決定，那大家對於將來缺電就不要埋怨，要對缺電作應有的決定。
對此，王美花回應表示，三接外推方案有很多的配套，包括氣源調度問題，包括大潭8、9號機如期完工，以高效率的機組取代老舊機組，經濟部與台電一定會確定國內能夠穩定供電。
另外，針對重啟核四的供電議題，王美花指出，核四的安全是第一個最大的問題，即使重啟也是11年後的事情，無法因應國內短期供電需求，經濟部基於支持台灣的經濟發展的立場，一定會確保國內穩定供電，也會儘快與台電討論核四資產最大化的議題。 
台電也進一步說明，核四重啟至少十年，本就不是短期選項。至於三接外推延後的兩年半，會影響到後年起112年至113年的供氣，台電會持續讓大潭8、9號機組如期完工，將現有氣源優先供氣給較高效率的燃氣機組，以提高全廠總發電量。
----------------------------------------------------------------------

[59] 44.3%民眾預期薪資上漲　逾5成估年終將上看3個月！

▲逾5成民眾樂觀看待年終最高上看3個月。（示意圖／pixabay）
記者陳依旻／台北報導
Omicron影響民眾對景氣的信心，國泰金最新國民經濟信心調查結果出爐！景氣展望、現況、民眾薪資上漲預期、大額消費意願這4個指數紛紛呈現下滑，但在薪資方面，預期會加薪的民眾比例從27.7%上揚至44.3%，54%民眾預期年終獎金最高3個月。
國泰金表示，國發會最新公布的10月景氣對策信號維持紅燈，領先指標與同步指標皆上揚；近期變種病毒開始傳播至全球多國，邊境管制趨嚴，景氣展望指數從11月的21.2下降

▲信義居家分別針對窗戶、微波爐以及浴室，提出6項懶人清潔小撇步。（圖／信義居家提供）
房產中心／綜合報導
《地產詹哥老實說》S2EP42／2022買房「站在高崗上」？網憂買到套到　專家曝最佳買點
首先針對窗戶清潔，專家表示，只要將報紙靠在紗窗上噴水讓它濕透、靜置五分鐘後撕除，即可將大部分的灰塵黏下來。關於窗軌死角，則可以用削尖的筷子或鉛筆，將灰塵撈出來，再準備一張名片，將它折成貼合窗軌的大小，之後拿出不要的化妝刷、牙刷、烤肉刷，先掃再擦。
▲只要將報紙靠在紗窗上噴水讓它濕透、靜置五分鐘後撕除，即可將大部分的灰塵黏下來。（圖／信義居家提供）
再來，微波爐的清潔，專家建議，可以放1條濕抹布在微波爐中加熱，微波過程中利用熱氣軟化污垢，再用濕抹布擦、就可以將這些污垢清除。若爐內實在太髒，也可以在可加熱的容器裡裝水，並放入檸檬切片、加熱1分鐘；加熱後不要急著拿出來，等個3、5分鐘讓熱氣把污垢溶解，再用抹布沾溫熱的檸檬水擦拭爐內，還可以有效去除微波裡惱人的氣味。
▲若微波爐內太髒，可以在可加熱的容器裡裝水，並放入檸檬切片、加熱1分鐘。（圖／信義居家提供）
專家提到，浴廁門通常都會有通風葉扇，葉扇的死角多，容易藏污髒垢，用抹布擦拭常常擦不到裡面的縫隙，針對這些細小區域的處理，一定是需要利用小工具才能快速的清潔。專家建議，可以先將方形海綿對角切割成直角三角形，由此一來即能方便又快速地清潔葉扇。
▲浴廁門通常都會有通風葉扇，葉扇的死角多，容易藏污髒垢。（圖／信義居家提供）
而水龍頭總是很快就產生灰灰的水垢，尤其在底部也常會因為積水而產生黏滑的黑漬。專家建議，可利用「小蘇打」或「牙膏」的研磨作用，進行清潔。
另外，水龍頭與檯面的接縫處，可以抹一點牙膏在破掉的「舊絲襪」上，鉤在水龍頭上面像洗澡魯拉拉的時候一樣，一拉一扯，絲襪馬上就會帶走縫中的髒污。
▲絲襪上抹牙膏，鉤在水龍頭上面，能帶走縫中的髒污。（圖／信義居家提供）
至於馬桶清潔，專家建案，可將寶特瓶底部鑽幾個小孔裝入白醋，放入馬桶水箱內，由此一來，每次沖水時，都能讓白醋沖洗到馬桶內壁，有除臭及清潔雙重的效果。
▲清潔劑各有不同功效，勿隨意將其混合在一起，嚴重的話可能產生有毒氣體，造成灼傷或是過敏。（圖／信義居家提供）
最後，專家提醒4個大掃除時的應注意事項，分別為「清潔劑遠離火源」、「清潔劑勿隨意混合」、「爬高應注意梯子的正確使用」

▲今年通膨造成物價明顯上漲，許多業者也因此宣布調漲。（示意圖／資料照）
記者高兆麟／綜合報導
行政院10月14日核定明年基本工資調漲方案，明年1月起基本薪資調漲約5.21%，基本月薪從24000元調漲至25250元、時薪由160元調至168元，同時經濟部也表示，基本工資補貼方案最快明年一月中旬開始受理，其中也包含受疫情影響的服務業及餐飲業，但民眾不滿許多餐飲業者在補助上路前就宣布漲價，等同「先漲價、再領補助」，怒嗆漲價店家不要領補助。
由於疫情三級警戒重創餐飲業及服務業，基本工資調漲又讓餐飲業營運壓力變得更為巨大，甚至可能打亂相關產業復甦，因此行政院考量今年度疫情衝擊國內內需產業，加上明年調漲基本工資對業主造成的負擔，11月4日拍板定案基本工資調漲的補貼配套方案，補助時期為明年1至6月，補貼對象包括受疫情影響的服務業、農業與部分製造業。
不過這個補貼方案還沒開始實施，就已經有不少餐飲業宣布調漲，包括如味丹「味味一品」泡麵、連鎖速食店如麥當勞、肯德基、丹丹漢堡等，以及如瓦城、八方雲集、梁社漢排骨等連鎖餐飲都宣布調漲，造成民眾不滿，認為這些業者好處佔盡，「一方面對消費者漲價，另一方面在跟政府拿補助」，讓消費者認為非常不公平，怒嗆這些漲價店家不要領補助。
商總和消基會也建議政府，應該要嚴查「無正當原因趁機哄抬物價」的不肖業者，如果查證屬實應要排除這類業者領取補貼，而行政院跨部會物價聯合稽查小組本周起也將全面展開稽查。
中華民國全國商業總會理事長許舒博表示，有些業者漲價是真的因應國際物價上漲，但國內上游並無明顯價格波動，卻有業者也漲價，政府應該要查清楚是「境外通膨」還是「境內通膨」調漲，而不是放任業者哄抬，如果國內有生產的原物料ㄝ沒遇到天災，也沒遇到生產不足，業者卻蓄意漲價，政府不但要查，嚴重的還可以取消補貼。
有民眾也認為，這些企業拿的「補助」，實際上也是拿老百姓的錢去補貼，這些業者一手拿老百姓納稅錢的補助，一手又向他們漲價，實在很扯，要這些已經漲價的業者，明年不要喊窮申請補助。
----------------------------------------------------------------------

[81] 金管會推虛擬幣管理新制　業者：樂見規範明確「保護幣圈新手」

▲隨著虛擬貨幣交易量大增，進而衍伸許多社會治安事件。（圖／記者邱中岳翻攝）
記者

▲財信傳媒董事長謝金河認為公投結果對台灣經濟有正面意義。（圖／記者湯興漢攝）
記者陳俐穎／綜合報導
四大公投落幕，四項議題全數遭到否決，而財信傳媒董事長謝金河也在臉書貼文發表意見，他認為，原先執政黨被認為不太樂觀，尤其是萊豬這一項很可能慘敗，不過，他相信台灣人民的集體意志的抉擇，每次都會為台灣找到新的出路，結果符合預期。謝金河認為，這次公投對台灣未來經濟發展及重返國際社會，會帶來十分正向的力量，股市的能量也會上升，人均所得也將在今年或明年超過南韓。
謝金河表示，每一次選舉都是決定台灣未來方向的溫度計，對每一次選舉都下足了功夫研究，大致上，命中率都很高，最意外的一次是2018年的韓流旋風席捲選局。這次公投，很多人都說執政黨不樂觀，尤其是萊豬這一項很可能慘敗。不過，他認為台灣人民的集體意志的抉擇，每次都會為台灣找到新的出路，因此覺得四個不同意可能會勝出，結果符合我的預期。
謝金河坦言，這次的公投釐清了盤踞在天空上的烏雲，對台灣未來經濟發展及重返國際社會，會帶來十分正向的力量，股市的能量也會上升，值得關注的是，台灣經濟下一個回合的脫胎換骨。
他提出，首先這兩年台灣經濟力道追上南韓，除了經濟成長率超過南韓，人均所得也將在今年或明年超過南韓。過去，台灣樣樣都不如南韓，大家都很洩氣，如今台灣正準備快速超車，這對台灣人民來說是不可思議的變化。二是，日本已經著手研究，台灣人均所得很可能在未來六到八年追上日本，這更是難以想像的事。三是台灣企業全面國際化，台積電到美國，日本，甚至到德國建廠，在過去，這也是天荒夜譚的事，未來除了台積電，會有更多企業走向世界。四是，台灣若能加入CPTPP或台美BTA，未來格局會更大。
另政治角度分析，他認為這次公投也是藍綠基本盤的溫度計，從公投結果來看，外島的金門，馬祖，及花蓮，台東，新竹縣，苗栗縣，南投縣，都是藍營鐵票區，下次選舉，藍營會全拿，台北市也是藍大於綠，下次國民黨拿回台北市機率比較大。這當中值得民進黨注意的是，沒有鄭文燦的桃園市長，沒有林右昌的基隆市長很可能都會輸掉。而雲林，彰化縣，綠營基本盤上升，下次縣長選舉，綠營只要提對人選，獲勝機率很大。
從這次公投結果來看，謝金河認為，彰化以南，綠營基本盤逐漸具有優勢。這次新北，一個不同意，三個同意，票數十分接近；台中市只有在公投綁大選一項，同意票些微勝出，藍綠在伯仲之間，這也可能使元月九日的台中二選區

In [21]:
browser.quit()